<a href="https://colab.research.google.com/github/meiqingli/Geospatial-Software-Design/blob/master/D_Lab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Time-Series Satellite Images with Google Earth Engine Python API

by: Meiqing Li (meiqing@berkeley.edu), D-Lab Data Science Fellow

This notebook demonstrates the process to filter, classify, resample, and visualiza time-series satellite images using Google Earth Engine Python API. 

### API Setup: import, authenticate and initialize

Beofre running the code, we need to first import the Google Earth Enginer API. Since the Earth Engine API is installed by default in Google Colaboratory, I don't need to install it here. 

In [1]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qLJqTz7e1xya_2XxnnXNOYOd19x75uNS4xV65Qj-_dc&tc=w_SKdMI0xNgCwZTKf2FcnZFpO403BepMDE8OqV8KiV0&cc=RRnSUS8gdK-6IK0sK3E1f_mPCqjQPN8PYoZ2z0YUguQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7I-T7FosUDVuxuQzUfKBbppR4PKV5rYjiFjlenmrLUVT48pZ5d7YA

Successfully saved authorization token.


### Explore Earth Engine Data Catalog

[The Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) 

In [3]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


https://code.earthengine.google.com/1ec1d414a4c5b03bbb0a7d6c19b5a748
https://code.earthengine.google.com/291961fe8406ca4c75ecea4ef3f33435

VIIRS - Nighttime Light Image


In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


### Import Features

In [13]:
# I have uploaded CA county boundaries through GEE console (add more description here)
CACounty = ee.FeatureCollection("projects/spherical-depth-278922/assets/California_County_Boundaries")
print(CACounty)

<bound method Collection.getInfo of <ee.featurecollection.FeatureCollection object at 0x7fe90fe24400>>


In [11]:
## night Lighting in 2013
NightImage = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182013')
# print(NightImage)

In [20]:
## check different bands - select band 1
NightImage = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182013').expression('b(1)')
# print(NightImage)

In [16]:
## Get statewide geometry for later use 
statewideGEOMETRY = CACounty.geometry()
# print(statewideGEOMETRY)

In [21]:
## Clip the images to sample areas
NightImage_CA = NightImage.clip(CACounty)

In [ ]:
## Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

## Paint all the polygon edges with the same number and width, display.
outline = empty.paint({featureCollection: CACounty, color: 1, width: 2})
Map.addLayer(outline, {palette: 'FFFFFF', opacity: 0.5}, 'CA Counties')

## Map.addLayer(CACounty,{color: '#EFC5C5', opacity: 0.1 },'CA Counties')
Map.addLayer(NightImage, {min: 0, max: 63, palette: ['000044','ffff00','ffffff'], opacity: 0.5},'Stable Light')
Map.centerObject(CACounty, 6);
county = CACounty.first()
county = ee.Feature(county)
print('One County Feature',county)

## Static image

In [72]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global nighttime light.
Image(url = NightImage.updateMask(NightImage.gt(0))
  .getThumbURL({'min': 0, 'max': 63, 'region': CACounty.filterMetadata("COUNTY_NAM","equals", "Contra Costa").first().geometry(), 'dimensions': 512,
                'palette': ['000044','ffff00','ffffff']}))

## Interactive map

In [47]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 63,
  'palette': ['000044','ffff00','ffffff']}

# Create a folium map object.
my_map = folium.Map(location=[38, -121], zoom_start=5.5)

# Add the CA county boundary to the map object.
my_map.add_ee_layer(CACounty.updateMask(CACounty.gt(0)), 'CA County')

# Add the nighttime light to the map object.
my_map.add_ee_layer(NightImage_CA.updateMask(NightImage_CA.gt(0)), vis_params, 'NightImage')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

AttributeError: ignored

### DMSP/OLS Nighttime Lights


### VIIRS Nighttime Light

### References

Google Earth Engine Python API: 
https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb

https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/intro-to-python-api/index.ipynb

https://developers.google.com/earth-engine/datasets/catalog
